<a href="https://colab.research.google.com/github/AsadiAhmad/ALU-8Bit/blob/main/Code/Word_Predictor_N_gram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Import Libraries

In [28]:
import requests
import polars as pl

# Step 2: Get Corpus

In [29]:
corpus_url = "https://raw.githubusercontent.com/AsadiAhmad/Word-Predictor/main/Corpus/Tarzan.txt"

response = requests.get(corpus_url)

if response.status_code == 200:
    raw_text = response.text
    print("Text fetched successfully!")
else:
    print(f"Failed to fetch the text. Status code: {response.status_code}")

Text fetched successfully!


# Step 3: Convert Corpus to Polars Data frame

## Extract paragraphs

In [30]:
paragraphs = raw_text.split("\r\n\r\n")

text = pl.DataFrame({
    "index": range(0, len(paragraphs)),
    "text": paragraphs
})

## Set Confige

In [31]:
pl.Config.set_tbl_rows(-1)
pl.Config.set_tbl_cols(-1)
pl.Config.set_fmt_str_lengths(100)

polars.config.Config

## Show parts of corpus

In [32]:
print(repr(raw_text))

'\ufeffThe Project Gutenberg eBook of Tarzan, lord of the jungle\r\n    \r\nThis ebook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever. You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this ebook or online\r\nat www.gutenberg.org. If you are not located in the United States,\r\nyou will have to check the laws of the country where you are located\r\nbefore using this eBook.\r\n\r\nTitle: Tarzan, lord of the jungle\r\n\r\n\r\nAuthor: Edgar Rice Burroughs\r\n\r\nRelease date: February 12, 2024 [eBook #72938]\r\n\r\nLanguage: English\r\n\r\nOriginal publication: New York, NY: Grosset & Dunlap, Publishers, 1928\r\n\r\nCredits: Greg Weeks, Mary Meehan and the Online Distributed Proofreading Team at http://www.pgdp.net\r\n\r\n\r\n*** START OF THE PROJECT GUTENBERG EBOOK TARZAN, LORD OF THE JUNGLE ***\r\n\r\n\r\n\r\n\r\n\r\n           

In [33]:
text.head(10)

index,text
i64,str
0,"""﻿The Project Gutenberg eBook of Tarzan, lord of the jungle This ebook is for the use of anyon…"
1,"""Title: Tarzan, lord of the jungle"""
2,""" Author: Edgar Rice Burroughs"""
3,"""Release date: February 12, 2024 [eBook #72938]"""
4,"""Language: English"""
5,"""Original publication: New York, NY: Grosset & Dunlap, Publishers, 1928"""
6,"""Credits: Greg Weeks, Mary Meehan and the Online Distributed Proofreading Team at http://www.pgdp.net"""
7,""" *** START OF THE PROJECT GUTENBERG EBOOK TARZAN, LORD OF THE JUNGLE ***"""
8,""""""


# Step 4: Preprocessing

## Clear some rows

In [26]:
text_filtered = text.slice(10, len(text))

In [27]:
text_filtered.head(10)

line_number,text
i64,str
11,""" Tarzan"""
12,""" LORD OF THE JUNGLE"""
13,""" By EDGAR RICE BURROUGHS"""
14,""" GROSSET & DUNLAP, _Publishers_, NEW YORK"""
15,""" Copyright 1928 by Edgar Rice Burroughs, Inc."""
16,""" _Copyright renewed 1956 by John Coleman Burroughs_"""
17,""" _All Rights Reserved_"""
18,""" _Printed in the United States of America_"""
19,""""""
